In [217]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

In [218]:
file = "Telco.csv"
df_telco = pd.read_csv(file)
df_telco.head()



,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [219]:
yes_no_columns = ["Partner", "Dependents", "PhoneService", "OnlineSecurity", "OnlineBackup", "DeviceProtection", "TechSupport", "StreamingTV", "StreamingMovies", "PaperlessBilling", "Churn"]
for item in yes_no_columns:
    df_telco[item] = df_telco[item].map({'Yes': 1, 'No': 0}).astype("Int64")

#Assume that if the total charges are null, replace with zeros
df_telco["TotalCharges"] = pd.to_numeric(df_telco["TotalCharges"], errors='coerce')
df_telco.fillna(0, inplace=True)
df_telco.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,1,0,1,0,No phone service,DSL,0,...,0,0,0,0,Month-to-month,1,Electronic check,29.85,29.85,0
1,5575-GNVDE,Male,0,0,0,34,1,No,DSL,1,...,1,0,0,0,One year,0,Mailed check,56.95,1889.50,0
2,3668-QPYBK,Male,0,0,0,2,1,No,DSL,1,...,0,0,0,0,Month-to-month,1,Mailed check,53.85,108.15,1
3,7795-CFOCW,Male,0,0,0,45,0,No phone service,DSL,1,...,1,1,0,0,One year,0,Bank transfer (automatic),42.30,1840.75,0
4,9237-HQITU,Female,0,0,0,2,1,No,Fiber optic,0,...,0,0,0,0,Month-to-month,1,Electronic check,70.70,151.65,1


In [220]:
y = df_telco["Churn"]
y = y.astype("int64")
X = df_telco.drop(["Churn", "customerID"], axis=1)
X.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,Female,0,1,0,1,0,No phone service,DSL,0,1,0,0,0,0,Month-to-month,1,Electronic check,29.85,29.85
1,Male,0,0,0,34,1,No,DSL,1,0,1,0,0,0,One year,0,Mailed check,56.95,1889.50
2,Male,0,0,0,2,1,No,DSL,1,1,0,0,0,0,Month-to-month,1,Mailed check,53.85,108.15
3,Male,0,0,0,45,0,No phone service,DSL,1,0,1,1,0,0,One year,0,Bank transfer (automatic),42.30,1840.75
4,Female,0,0,0,2,1,No,Fiber optic,0,0,0,0,0,0,Month-to-month,1,Electronic check,70.70,151.65


In [221]:
category_columns = ["gender", "MultipleLines", "InternetService", "Contract", "PaymentMethod"]
X = pd.get_dummies(X, columns=category_columns)
X.replace([np.inf, -np.inf], 0, inplace=True)
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 29 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   SeniorCitizen                            7043 non-null   int64  
 1   Partner                                  7043 non-null   Int64  
 2   Dependents                               7043 non-null   Int64  
 3   tenure                                   7043 non-null   int64  
 4   PhoneService                             7043 non-null   Int64  
 5   OnlineSecurity                           7043 non-null   Int64  
 6   OnlineBackup                             7043 non-null   Int64  
 7   DeviceProtection                         7043 non-null   Int64  
 8   TechSupport                              7043 non-null   Int64  
 9   StreamingTV                              7043 non-null   Int64  
 10  StreamingMovies                          7043 no

In [222]:
# Hyperparameters
random_state = 42
n_estimators = 500
max_depth = 4
max_features = 3
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .25, random_state = 42)
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [223]:
mdl = RandomForestClassifier(n_estimators = n_estimators, max_depth = max_depth, max_features = max_features, bootstrap = True, random_state = random_state).fit(X_train, y_train)
mdl.fit(X_train, y_train)

RandomForestClassifier(max_depth=4, max_features=3, n_estimators=500,
                       random_state=42)

In [224]:
ypred_train = mdl.predict(X_train)
ypred_test = mdl.predict(X_test)
results = [f1_score(y_train, ypred_train),
                                f1_score(y_test, ypred_test)]
results

[0.43969465648854955, 0.43004418262150224]

In [225]:
params = {"max_depth": [2,5,10,15,30], "n_estimators" : [30,50,100,250,500]}
grid_search = GridSearchCV(RandomForestClassifier(random_state=42), params, cv = 5,n_jobs = -1)
grid_search.fit(X, y)
grid_search.best_estimator_

RandomForestClassifier(max_depth=10, random_state=42)

In [226]:
grid_search.best_params_

{'max_depth': 10, 'n_estimators': 100}

In [227]:
mdl = GradientBoostingClassifier(learning_rate = 0.1,
                                    n_estimators=250,
                                    max_depth = 3,
                                    random_state = 42)
mdl.fit(X_train, y_train)

ypred_train = mdl.predict(X_train)
ypred_test = mdl.predict(X_test)
results = [f1_score(y_train, ypred_train), f1_score(y_test, ypred_test)]
results

[0.6874251497005989, 0.5851938895417156]

In [228]:
params = {"max_depth": [2,5,10,15,30], "n_estimators" : [30,50,100,250,500],
        "learning_rate": [1e-4,1e-3,1e-2,0.1,0.5]}
grid_search = GridSearchCV(GradientBoostingClassifier(random_state=42),params, cv = 5, n_jobs=-1)
grid_search.fit(X_train, y_train)
grid_search.best_estimator_
grid_search.best_params_